In [1]:
print("""
    In oder to build the therapist I can think of two solution.
    1. Given the query or speech by user A: we can try to understand the tone or emotion in the speech or text, for that we can use either LLMs, to
    find the emotion or can fine-tune our own emotion detection (which is done by me in the emotion detection folder). 
    2. Once we know if person is suffering from anger management issue or depression we can perform the chatting 
    with the corresponding Agent type: for example Agent therapist expert in depression handling, or Agent therapise expert in anger handling.
    3. We can also use the same Agent expert in both. 

    4. Another approach could be find the emotion of a person, based on speech or text. and than pull out the correspoinding informations based on
    the certain problem for example if person suffering from anger issue we will have the dataset for handling the anger issue created or framed 
    by actual therapist, than we can create a RAG system which will help us answer the user query. 

    5. To handle the Hallucination we can grade the responses generated from LLMs. 
    6. We can also handle the case when the agent unable to answer in that case we can move to search online from internet to generate the 
    answer. 
""")


    In oder to build the therapist I can think of two solution.
    1. Given the query or speech by user A: we can try to understand the tone or emotion in the speech or text, for that we can use either LLMs, to
    find the emotion or can fine-tune our own emotion detection (which is done by me in the emotion detection folder). 
    2. Once we know if person is suffering from anger management issue or depression we can perform the chatting 
    with the corresponding Agent type: for example Agent therapist expert in depression handling, or Agent therapise expert in anger handling.
    3. We can also use the same Agent expert in both. 

    4. Another approach could be find the emotion of a person, based on speech or text. and than pull out the correspoinding informations based on
    the certain problem for example if person suffering from anger issue we will have the dataset for handling the anger issue created or framed 
    by actual therapist, than we can create a RAG system whic

# Import dataset.

In [2]:
import datasets
data = datasets.load_dataset("Amod/mental_health_counseling_conversations")

/Users/apple/miniforge3/envs/env-langchain/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
all_responses = []
for train in data["train"]:
    all_responses.append(train["Response"])
mental_health = " ".join(all_responses)

## Let's use langgraph to tackle this issue.

In [4]:
local_llm = "mistral"

Defining the two different retriver
* For anger.
* For depression.

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.docstore.document import Document

doc = Document(page_content=mental_health,metadata={"source": "local"})
# retreival system 
embeddings = GPT4AllEmbeddings()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=256, chunk_overlap=0
)

splitted_docs = text_splitter.split_documents([doc])
# let's compute the embeddings 

vectorstore = Chroma.from_documents(
    documents=splitted_docs,
    collection_name="rag-therapist-depression",
    embedding=GPT4AllEmbeddings(),
)
retriever_depression = vectorstore.as_retriever(search_kwargs={"k": 5})

In [6]:
from langchain_community.document_loaders import WebBaseLoader

urls = ["https://www.apa.org/topics/anger/control"]
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]


embeddings = GPT4AllEmbeddings()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=256, chunk_overlap=0
)
splitted_docs = text_splitter.split_documents(docs_list)
# let's compute the embeddings 
vectorstore = Chroma.from_documents(
    documents=splitted_docs,
    collection_name="rag-therapist-anger",
    embedding=GPT4AllEmbeddings(),
)
retriever_angry= vectorstore.as_retriever(search_kwargs={"k": 5})


In [61]:
mapping_to_retriever = {
    "depression":  retriever_depression,
    "angry": retriever_angry
}

In [11]:
# finding the type of emotion 
# we can also use the model i have trained inside the emotion detection folder
# this model can be used as a tool, we can create a structured tool to use that model which return the type of emotion.
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import JsonOutputParser
# we need to udnerstand what kind of emotion is in query
llm = ChatOllama(model=local_llm, format="json", temperature=0)
prompt="""
    Your an expert in understanding the emotion in the query, provided to you by the user. 
    Your task is to understand the query and assign the emotion based on your semantic understanding, the labels that can be assigned to the queries are:
    1. angry
    2. depression
    3. otherwise
    Do not generate any other emotion apart from what is being suggested to you.
    Query to assign type of emotion: {Query}, 
    Provide the respose in Json format so key is going to be emotion and value is type of emotion, do not provide any other kind of explaination.
"""
template = PromptTemplate.from_template(prompt)
emotion_detection = template | llm | JsonOutputParser()
emotion_detection.invoke({"Query": "Shit hit the fence i need money!"})

{'emotion': 'angry'}

In [65]:
# this is an LLM based Agent which can handled for the cases where person don't know 
# if it's depressed or angry like the person has no information about it's condition
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
llm = ChatOllama(model=local_llm, temperature=0)
PROMPT = """ As a trained psychologist specialized in emotional well-being, your task is to support individuals facing various challenges in life, including but not limited to:

1. Anger Management
2. Depression
3. Relationship Breakups
4. Anxiety
5. Family Conflicts

The aim is to be a compassionate companion on this journey towards healing and self-discovery, providing practical solutions to navigate these difficult times.

When addressing concerns, emphasis will be placed on:

- Meditation and mindfulness practices for mental clarity
- Spirituality for inner peace and resilience
- Focusing on factors within one's control to foster happiness and reduce worry

if the input from the user is not related to above discussed point you can do the normal conversation.

You will be provided with:
Current conversation:
{history}
Human: {input}
AI Assistance:
"""

conversation = ConversationChain(
    llm = llm,
    prompt = PromptTemplate(input_variables=["history","input"],template=PROMPT),
    verbose=False,
)

In [70]:
conversation.invoke({"input":"hello","history":[""]})

{'input': 'hello',
 'history': "Human: hello\nAI:  Hello! I'm here to support you in various aspects of emotional well-being, including anger management, depression, relationship breakups, anxiety, and family conflicts. If you're going through any of these issues or something similar, please feel free to share and we can work together towards healing and self-discovery.\n\nHowever, if your concern is not related to the above points, I'd be happy to engage in a normal conversation as well! What's on your mind today?",
 'response': " Hello there! I'm here to support you in various aspects of emotional well-being, including anger management, depression, relationship breakups, anxiety, and family conflicts. If you're going through any of these issues or something similar, please feel free to share and we can work together towards healing and self-discovery.\n\nHowever, if your concern is not related to the above points, I'd be happy to engage in a normal conversation as well! What's on you

In [66]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
template = """
    You help everyone by answering questions, and improve your answers from previous answers in History.
    Don't try to make up an answer, if you don't know, just say that you don't know.
    Answer in the same language the question was asked.
    Answer in a way that is easy to understand.
    Do not say "Based on the information you provided, ..." or "I think the answer is...". Just answer the question directly in detail.

    History: {chat_history}

    Context: {context}

    Question: {question}
    Answer: 
"""

prompt = ChatPromptTemplate.from_template(template)
rag_chain = prompt | llm | StrOutputParser()


In [67]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain_community.document_compressors.llmlingua_filter import LLMLinguaCompressor
from langchain_core.runnables import RunnablePassthrough
from langchain import hub

compressor = LLMLinguaCompressor(model_name="openai-community/gpt2", device_map="cpu")

In [85]:
from typing_extensions import TypedDict
from typing import List

class AgentState(TypedDict):
    question: str
    response: str
    documents: List[str]
    chat_history_chat_agent: List[str]
    chat_history: List[str]
    
# get the emotion of the queries by the user
def getEmotions(state):
    question = state["question"]
    type_of_emotion = emotion_detection.invoke({"Query": question})

    print(type_of_emotion)
    if type_of_emotion["emotion"] == "angry":
        # we need to perform one type of retrieval which take care of angry part
        return "angry"
    elif type_of_emotion["emotion"] == "depression":
        # we need to route to another type of retreival 
        return "depression"
    else:
        return "chatwithagent"

def depression_retreival(state):
    question= state["question"]
    # given these documents we can also do much more than just retreiving the data
    # we can perform contextual compression 
    # we can do re-ranking of the documents
    compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever_depression
        )
    compressed_docs = compression_retriever.get_relevant_documents(question)
    history = [doc.page_content for doc in compressed_docs]
    return {"question": question, "documents": compressed_docs, "documents": history}


def anger_retreival(state):
    question= state["question"]
    # given these documents we can also do much more than just retreiving the data
    # we can perform contextual compression 
    # we can do re-ranking of the documents
    compression_retriever = ContextualCompressionRetriever(
            base_compressor=compressor, base_retriever=retriever_angry
        )
    compressed_docs = compression_retriever.get_relevant_documents(question)
    history = [doc.page_content for doc in compressed_docs]
    return {"question": question, "documents": history}

def chat_with_agent(state):
    question = state["question"]
    chat_history_chat_agent = state["chat_history_chat_agent"]
    if chat_history_chat_agent == None:
        chat_history_chat_agent = [""]
    response = conversation.invoke({"input": question, "history": chat_history_chat_agent })

    if len(chat_history_chat_agent) > 10:
        del chat_history_chat_agent[0]
        chat_history_chat_agent.append(response["history"])
    else:
        chat_history_chat_agent.append(response["history"])

    return {"question": question, "response": response["response"], "chat_history_chat_agent":chat_history_chat_agent}

def generate_response(state):
    question = state["question"]
    context = state["documents"]
    chat_history = state["chat_history"]
    response = rag_chain.invoke({"question": question, "context": context, "chat_history": chat_history})
    if chat_history == None:
        chat_history = [response]
    elif len(chat_history) > 10:
        del chat_history[0]
        chat_history.append(response)
    
    return {"question": question, "response": response, "chat_history": chat_history}


def end_of_conversation(state):
    return "end"

# creating the workflow.

In [86]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(AgentState)

workflow.add_node("depression_retreival", depression_retreival)
workflow.add_node("anger_retreival", anger_retreival)
workflow.add_node("chat_with_agent", chat_with_agent)
workflow.add_node("generate_response",generate_response)

workflow.set_conditional_entry_point(
    getEmotions,
    {
        "angry": "anger_retreival",
        "depression": "depression_retreival",
        "chatwithagent": "chat_with_agent"
    }
)

workflow.add_edge("anger_retreival", "generate_response")
workflow.add_edge("depression_retreival", "generate_response")
workflow.add_conditional_edges(
    "chat_with_agent",
    end_of_conversation,
    {
        "end": END
    }
)

workflow.add_conditional_edges(
    "generate_response",
    end_of_conversation,
    {
        "end": END
    }
)


In [87]:
app = workflow.compile()

In [88]:
from pprint import pprint
# Run 
while True:
    question = input("Enter the question")
    if question.lower() =="exit":
        break
    inputs = {"question": question}
    for output in app.stream(inputs):
        for key, value in output.items():
            pprint(f"Node '{key}':")
        pprint("\n---\n")

    print(value["response"])


{'emotion': 'otherwise'}
"Node 'chat_with_agent':"
'\n---\n'
 Hello! I'm here to support you in various aspects of emotional well-being, including anger management, depression, relationship breakups, anxiety, and family conflicts. If you're going through any of these issues or something similar, please feel free to share and we can work together towards healing and self-discovery.

However, if your concern is not related to the above points, I'd be happy to engage in a normal conversation as well! Is there a particular topic you'd like to discuss or any question you have that I might be able to help answer? Let me know and we can explore it together. For instance, we could talk about hobbies, books, movies, or even share some jokes to lighten the mood. What would you prefer?

If you don't have a specific preference, I can suggest some topics based on your interests if you'd like! Let me know how I can be of help.

Additionally, I can recommend meditation and mindfulness practices for m

Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{'emotion': 'angry'}
"Node 'anger_retreival':"
'\n---\n'
"Node 'generate_response':"
'\n---\n'
 I'm sorry to hear that you're feeling angry. It's natural to experience emotions from time to time. If you need someone to talk to or if there's a specific issue that's causing your anger, feel free to share it with me. I'll do my best to help you out or provide some resources for you to explore further. In the meantime, taking deep breaths and engaging in calming activities such as meditation or exercise can help manage feelings of anger.


Number of requested results 5 is greater than number of elements in index 1, updating n_results = 1


{'emotion': 'angry'}
"Node 'anger_retreival':"
'\n---\n'
"Node 'generate_response':"
'\n---\n'
 There are several effective ways to help reduce anger:

1. Practice relaxation techniques: Deep breathing, progressive muscle relaxation, and meditation are all proven methods for reducing stress and anger.
2. Identify triggers: Keep a journal to identify situations or people that trigger your anger. Once you've identified these triggers, you can take steps to avoid them or develop coping strategies.
3. Exercise regularly: Regular physical activity can help reduce stress and improve your mood.
4. Get enough sleep: Lack of sleep can make it more difficult to manage emotions and increase feelings of irritability and anger.
5. Eat a healthy diet: A healthy diet can help improve your mood and energy levels, making it easier to cope with stressors.
6. Practice positive thinking: Focus on the good things in your life and try to reframe negative thoughts into positive ones.
7. Seek support: Talking

In [164]:
value["response"]

' It\'s important to understand that problems cannot be "let go" but are resolved through recognition and patience. Trying to disconnect from reality every day isn\'t possible, and recognizing the causes of your problems can lead to peacefulness and resolution. Be patient with this process and allow yourself time for self-reflection.'

In [157]:
prompt = hub.pull("rlm/rag-prompt")

In [158]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])